<a target="_blank" href="https://colab.research.google.com/github/sudarshan-koirala/langchain-falcon-chainlit/blob/main/langchain_falcon.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
# USE CASE:    CONNECTED
#   TEXT  (YouTube Example = https://www.youtube.com/watch?v=gnyUUY8X-G4)   
# + VIDEO (https://huggingface.co/learn/audio-course/chapter6/pre-trained_models)

In [2]:
%%capture
%pip install langchain huggingface_hub watermark
%pip install transformers datasets soundfile speechbrain accelerate
%pip install sentencepiece
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [3]:
#%load_ext watermark
%reload_ext watermark
%watermark -a "Falcon FutureTech Maverick Team" -vmp langchain,huggingface_hub

Author: Falcon FutureTech Maverick Team

Python implementation: CPython
Python version       : 3.10.8
IPython version      : 8.16.1

langchain      : 0.0.324
huggingface_hub: 0.17.3

Compiler    : GCC 9.4.0
OS          : Linux
Release     : 6.2.0-1015-azure
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit



In [4]:
# get your Huggingface access token from https://huggingface.co/settings/tokens 🔑
from getpass import getpass
import os

HUGGINGFACE_API_TOKEN = getpass()
os.environ["HUGGINGFACE_API_TOKEN"] = HUGGINGFACE_API_TOKEN   

#### Let's use falcon-7b-instruct model from [Huggingface website](https://huggingface.co/tiiuae/falcon-7b-instruct)

In [5]:
from langchain import HuggingFaceHub

repo_id = "tiiuae/falcon-7b-instruct"
llm = HuggingFaceHub(huggingfacehub_api_token=HUGGINGFACE_API_TOKEN, 
                     repo_id=repo_id, 
                     model_kwargs={"temperature":0.7, "max_new_tokens":700})

/workspaces/falcon-main/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspaces/falcon-main/.venv/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [6]:
from langchain import PromptTemplate, LLMChain

template = """
You are a helpful AI assistant and provide the answer for the question asked politely.

{question}
Answer: Let's think step by step.
"""
prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "How to cook Pizza ?"

In [7]:
responseText = llm_chain.run(question)

len(responseText)

print(responseText)


1. Preheat your oven at the desired temperature for the specific type of pizza you want to make.
2. Roll out your dough on a baking sheet or pizza stone.
3. Spread your favorite pizza sauce on the dough.
4. Add your desired amount of cheese.
5. Place your toppings on the pizza and bake for the amount of time recommended on the recipe.
6. Remove the pizza from the oven and let it cool before slicing. 
7. Enjoy your delicious pizza!

I hope this helps!


In [8]:
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech

checkpoint = "microsoft/speecht5_tts"
processor = SpeechT5Processor.from_pretrained(checkpoint)
model = SpeechT5ForTextToSpeech.from_pretrained(checkpoint)

: 

In [ ]:
inputs = processor(text=responseText, return_tensors="pt")

print (inputs)

In [ ]:
# Let’s load such a speaker embedding from a dataset on the Hub. 

# the embeddings were obtained from the CMU ARCTIC dataset: 
# http://www.festvox.org/cmu_arctic/

# using this script: 
# https://huggingface.co/mechanicalsea/speecht5-vc/blob/main/manifest/utils/prep_cmu_arctic_spkemb.py

# but any X-Vector embedding should work.

from datasets import load_dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
len(embeddings_dataset)

In [ ]:
# The speaker embedding is a tensor of shape (1, 512). 
# This particular speaker embedding describes a female voice.
import torch

speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)
print(inputs["input_ids"])

In [ ]:
# At this point we already have enough inputs to generate a log mel spectrogram as an output, 
# you can do it like this:

spectrogram = model.generate_speech(inputs["input_ids"], speaker_embeddings)

In [ ]:
print(spectrogram)

# This outputs a tensor of shape (140, 80) containing a log mel spectrogram.

# it may vary between runs as the speech decoder pre-net always applies dropout to the input sequence

# This adds a bit of random variability to the generated speech

# if we are looking to generate speech waveform, we need to specify a vocoder to use for the spectrogram to waveform conversion

# Conveniently, 🤗 Transformers offers a vocoder based on HiFi-GAN

# HiFi-GAN is a state-of-the-art generative adversarial network (GAN) designed for high-fidelity speech synthesis. 
# It is capable of generating high-quality and realistic audio waveforms from spectrogram inputs.

In [ ]:
from transformers import SpeechT5HifiGan

vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
speech = model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)

In [ ]:
from IPython.display import Audio
Audio(speech, rate=16000)